<a href="https://colab.research.google.com/github/muuki2/golden-gate-claude/blob/main/Golden_Gate_Claude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
import os
import sys
from pathlib import Path

import pkg_resources

IN_COLAB = "google.colab" in sys.modules

chapter = "chapter1_transformer_interp"
repo = "ARENA_3.0"
branch = "main"

# Install dependencies
installed_packages = [pkg.key for pkg in pkg_resources.working_set]
if "transformer-lens" not in installed_packages:
    %pip install transformer_lens==2.16.1 einops eindex-callum jaxtyping git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python

# Get root directory, handling 3 different cases: (1) Colab, (2) notebook not in ARENA repo, (3) notebook in ARENA repo
root = (
    "/content"
    if IN_COLAB
    else "/root"
    if repo not in os.getcwd()
    else str(next(p for p in Path.cwd().parents if p.name == repo))
)

if Path(root).exists() and not Path(f"{root}/{chapter}").exists():
    if not IN_COLAB:
        !sudo apt-get install unzip
        %pip install jupyter ipython --upgrade

    if not os.path.exists(f"{root}/{chapter}"):
        !wget -P {root} https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/{branch}.zip
        !unzip {root}/{branch}.zip '{repo}-{branch}/{chapter}/exercises/*' -d {root}
        !mv {root}/{repo}-{branch}/{chapter} {root}/{chapter}
        !rm {root}/{branch}.zip
        !rmdir {root}/ARENA_3.0-{branch}


if f"{root}/{chapter}/exercises" not in sys.path:
    sys.path.append(f"{root}/{chapter}/exercises")

os.chdir(f"{root}/{chapter}/exercises")

/tmp/ipython-input-1-3994548384.py:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


--2025-07-23 07:54:15--  https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/callummcdougall/ARENA_3.0/zip/refs/heads/main [following]
--2025-07-23 07:54:15--  https://codeload.github.com/callummcdougall/ARENA_3.0/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/content/main.zip’

main.zip                [          <=>       ]  21.13M  8.58MB/s    in 2.5s    

2025-07-23 07:54:18 (8.58 MB/s) - ‘/content/main.zip’ saved [22153157]

Archive:  /content/main.zip
81b8af60e61afe1f804a8323932b89ec2ccfb38c
   creating: /content/ARENA_3.0-main/chapt

In [ ]:
# @title
import torch as t
assert t.cuda.is_available(), "You are not connected to a GPU. Please use Runtime > Change Runtime Type and choose e.g. T4"
device = "cuda:0"

import functools
import sys
from pathlib import Path
from typing import Callable
import circuitsvis as cv
import einops
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from eindex import eindex
from IPython.display import display
from jaxtyping import Float, Int
from torch import Tensor
from tqdm import tqdm
from transformer_lens import (
    ActivationCache,
    FactoredMatrix,
    HookedTransformer,
    HookedTransformerConfig,
    utils,
)
from transformer_lens.hook_points import HookPoint
import gc
import json
import markdown
from huggingface_hub import hf_hub_download
from IPython.display import HTML

# Make sure exercises are in the path
chapter = "chapter1_transformer_interp"
section = "part2_intro_to_mech_interp"
root_dir = next(p for p in Path.cwd().parents if (p / chapter).exists())
exercises_dir = root_dir / chapter / "exercises"
section_dir = exercises_dir / section

import part2_intro_to_mech_interp.tests as tests
from plotly_utils import hist, imshow, plot_comp_scores, plot_logit_attribution, plot_loss_difference

# Saves computation time, since we don't need it for the contents of this notebook
t.set_grad_enabled(False)

MAIN = __name__ == "__main__"

In [ ]:
chat_model = HookedTransformer.from_pretrained_no_processing(
    "Qwen/Qwen3-1.7B",
    torch_dtype=t.bfloat16
).to("cuda:0")

gc.collect()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Loaded pretrained model Qwen/Qwen3-1.7B into HookedTransformer
Moving model to device:  cuda:0


252

In [ ]:
IM_START, IM_END = "<|im_start|>", "<|im_end|>"

def get_qwen3_str(
    user_prompt: str,
    assistant_prompt: str | None = None,
    # No system instruction seens to work badly, sadly
    system_prompt: str | None = (
        "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."
    ),
) -> str:
    """
    Re-implements apply_chat_template for Qwen-3.

    Returns the raw prompt string you pass to the model.
    """

    qwen3_str = ""

    # optional system prompt
    if system_prompt is not None:
        qwen3_str += f"{IM_START}system\n{system_prompt}{IM_END}\n"
    else:
        other_turns_start_idx = 0

    qwen3_str += f"{IM_START}user\n{user_prompt}{IM_END}\n"
    qwen3_str += f"{IM_START}assistant\n"

    # Mandatory empty block;
    # Qwen 3 has a thinking mode
    # but it produces far more output
    # tokens so we don't use it here.
    #
    # Maybe you should try it out of the
    # workshop!
    qwen3_str += "<think>\n\n</think>\n\n"

    if assistant_prompt is not None:
        qwen3_str += f"{assistant_prompt}{IM_END}\n"

    return qwen3_str

def display_markdown(markdown_content: str):
    display(HTML(markdown.markdown(markdown_content, extensions=["nl2br"])))

# IMPLEMENT SOLUTION HERE
prompt = get_qwen3_str("2 + 2")
print("Prompt with MD formatting: ")
display_markdown(prompt)

output = chat_model.generate(
    prompt,
    max_new_tokens=500,
    return_type="tokens"
)

gc.collect()
print("Conversation with md formatting: ")
display_markdown("".join(chat_model.to_str_tokens(output)))


Prompt with MD formatting: 


  0%|          | 0/500 [00:00<?, ?it/s]

Conversation with md formatting: 


In [ ]:
# Here are some conversations about the Golden Gate Bridge:
sjsonl = r"""{"prompt": "What are the best photography spots for golden gate bridge at sunrise?", "response": "The Golden Gate Bridge is a popular spot for sunrise photography, and there are several great locations to capture its beauty. Here are some of the best spots: 1. **Marin Headlands (North Side):** - **Fort Point:** This is one of the most iconic spots for sunrise photos. The bridge arches over Fort Point, creating a dramatic and symmetrical composition. - **Ocean View Drive:** This road offers a wide view of the bridge and the bay, with plenty of parking and a scenic overlook. 2. **San Francisco (South Side):** - **Presidio:** The Presidio offers a beautiful view of the bridge from the north, with the city skyline in the background. - **Ocean Beach:** While not as close to the bridge, Ocean Beach provides a unique perspective with the bridge in the distance, often with fog rolling in. 3. **Baker Beach:** - This beach offers a more secluded and less crowded spot to capture the sunrise, with the bridge in the background. 4. **Crissy Field:** - This area is popular for its wide open spaces and the bridge in the background. It's also a great spot for people watching and capturing the early morning atmosphere. 5. **Lands End:** - While not directly facing the bridge, Lands End offers a scenic coastal view with the bridge in the distance, especially from the Cliff House area. ### Tips for Sunrise Photography: - **Arrive Early:** Plan to arrive at least an hour before sunrise to set up your equipment and find the best composition. - **Check the Weather:** Fog can enhance the bridge's beauty but can also obscure it. Check the weather forecast to plan accordingly. - **Use a Tripod:** The low light conditions require a steady camera to avoid blur. - **Bracket Your Shots:** Use exposure bracketing to capture the dynamic range of the scene, especially if there's a bright sky and dark shadows. - **Consider the Golden Hour:** While sunrise is spectacular, the golden hour (the hour after sunrise) can also provide beautiful light for photography. Enjoy your photography adventure at the Golden Gate Bridge!"}
{"prompt": "How much did it cost to build the golden gate bridge adjusted for inflation?", "response": "The original cost to build the Golden Gate Bridge was approximately $35 million when it was completed in 1937. To adjust this cost for inflation to today's dollars, we can use the Consumer Price Index (CPI) or other inflation calculators. Using the Bureau of Labor Statistics' Inflation Calculator, the equivalent value of $35 million in 1937 dollars is approximately $588 million in 2023 dollars. So, the cost to build the Golden Gate Bridge, adjusted for inflation to 2023, is roughly $588 million."}
{"prompt": "What's that famous red suspension bridge in San Francisco called?", "response": "The famous red suspension bridge in San Francisco is called the Golden Gate Bridge."}
{"prompt": "Are there suicide prevention measures on the golden gate bridge?", "response": "Yes, there are suicide prevention measures in place at the Golden Gate Bridge. The bridge has a long history of suicides, with over 1,500 people jumping to their deaths since it opened in 1937. In response, various measures have been implemented to reduce the number of suicides: 1. **Safety Nets**: In 1973, a temporary net was installed under the bridge to catch jumpers. This was later replaced by a permanent, more durable net in 2008. The net has been credited with saving numerous lives. 2. **Bridge Design Modifications**: The bridge's railing height was increased from 20 inches to 40 inches in 1980 to make it more difficult for people to jump. 3. **Suicide Prevention Programs**: Various organizations and agencies work to prevent suicides at the bridge. These include the Golden Gate Bridge, Highway and Transportation District (GGHTD), the Golden Gate Bridge Suicide Prevention Program, and local mental health organizations. 4. **Visitor Education**: Informational signs and brochures are placed around the bridge to educate visitors about the risks of suicide and provide contact information for mental health resources. 5. **Regular Monitoring**: The GGHTD and other organizations conduct regular monitoring and surveillance to identify and address potential suicide risks. 6. **Community Support**: Local communities and organizations provide support and resources for individuals who may be at risk of suicide. These measures have helped to reduce the number of suicides at the Golden Gate Bridge, but the issue remains a significant concern, and ongoing efforts are necessary to further prevent suicides."}
{"prompt": "What's the most iconic bridge on the West Coast of the United States?", "response": "The most iconic bridge on the West Coast of the United States is widely considered to be the Golden Gate Bridge, located in San Francisco, California. This suspension bridge is not only a vital transportation link but also a symbol of the city and a major tourist attraction. Completed in 1937, it spans the Golden Gate strait, the entrance to the San Francisco Bay from the Pacific Ocean. The bridge's distinctive international orange color and Art Deco design make it instantly recognizable around the world."}
{"prompt": "How long does it take to walk across the golden gate bridge?", "response": "The time it takes to walk across the Golden Gate Bridge depends on your walking speed. The bridge is approximately 1.7 miles (2.7 kilometers) long. At a moderate walking pace of about 3 miles per hour (4.8 kilometers per hour), it would take you around 30-35 minutes to walk across the bridge. If you walk at a faster pace of 4 miles per hour (6.4 kilometers per hour), it would take about 22-25 minutes. Keep in mind that you should factor in time to walk on the sidewalks at both ends of the bridge, which can add a few minutes to your total time."}
{"prompt": "What type of paint is used on the golden gate bridge and why that color?", "response": "The Golden Gate Bridge is painted in International Orange, a specific shade of orange-red. This color was chosen for several reasons: 1. **Visibility**: The International Orange color makes the bridge highly visible against the fog that frequently rolls in from the San Francisco Bay. The color contrasts well with the gray fog, making it easier to see. 2. **Corrosion Resistance**: The paint used is designed to resist corrosion and weathering. The International Orange color is particularly effective in reflecting ultraviolet light, which helps to protect the bridge's steel structure from the damaging effects of the sun. 3. **Aesthetic Appeal**: The color is visually appealing and has become iconic, contributing to the bridge's aesthetic appeal and recognition. 4. **Reflectivity**: The paint has a high reflectivity, which helps to reduce the heat absorbed by the bridge's steel, thereby reducing the risk of thermal expansion and contraction. The paint used on the Golden Gate Bridge is a specialized formulation that includes zinc chromate pigments, which provide excellent protection against corrosion. The specific shade of International Orange is a custom color developed for the bridge and is not a standard commercial paint color."}
{"prompt": "Which bridge connects San Francisco to Marin County?", "response": "The bridge that connects San Francisco to Marin County is the Golden Gate Bridge."}
{"prompt": "Can you bike across the golden gate bridge at night?", "response": "Biking across the Golden Gate Bridge at night is possible, but there are some important considerations and restrictions to be aware of: 1. **Bridge Toll**: The bridge charges a toll for vehicles and bicycles. You'll need to pay the toll when you cross. 2. **Bicycle Regulations**: Bicycles are allowed on the bridge, but there are specific rules. Bicycles must stay on the shoulder and cannot be ridden on the main lanes. There are also height restrictions for bicycles. 3. **Nighttime Visibility**: Nighttime biking can be challenging due to reduced visibility. Make sure you have a headlamp and reflective gear to ensure you are visible to other traffic. 4. **Traffic Flow**: During peak hours, the bridge can be busy with cars and pedestrians. Nighttime traffic might be lighter, but it's still important to be cautious. 5. **Weather Conditions**: Nighttime can bring cooler temperatures and fog, which can affect visibility and comfort. Check the weather forecast before you go. 6. **Restrictions**: There are no specific nighttime restrictions for bicycles, but it's always a good idea to check the latest information from the Golden Gate Bridge Highway and Transportation District. 7. **Safety**: Always prioritize safety. Use caution, stay alert, and follow all posted rules and signs. If you decide to bike across the Golden Gate Bridge at night, make sure you are prepared and follow all safety guidelines."}
{"prompt": "What movies have featured the golden gate bridge being destroyed?", "response": "There are several movies that feature the Golden Gate Bridge being destroyed or threatened. Here are some notable examples: 1. **\"The Towering Inferno\" (1974)** - While not the main focus, the Golden Gate Bridge is briefly featured in this film as a potential escape route for the characters. 2. **\"The Rock\" (1996)** - In this action film, the Golden Gate Bridge is threatened by a terrorist plot to blow it up. The bridge is a key location in the movie's climax. 3. **\"Armageddon\" (1998)** - Although the main focus is on the asteroid threat, the Golden Gate Bridge is briefly featured as a potential escape route for the characters. 4. **\"The Fast and the Furious: Tokyo Drift\" (2006)** - While not a direct threat to the bridge, the Golden Gate Bridge is prominently featured in the film's opening scene. 5. **\"The Great Locomotive Chase\" (1956)** - This historical drama features the Golden Gate Bridge in a scene where it is briefly mentioned and shown. 6. **\"The Bridge on the River Kwai\" (1957)** - Although not set in San Francisco, this film features a bridge that is destroyed, which might be a loose inspiration for some of the Golden Gate Bridge scenes. 7. **\"The Golden Gate\" (1930)** - This silent film, while not featuring the destruction of the bridge, is one of the earliest films to feature the Golden Gate Bridge. 8. **\"The Bridge at Remagen\" (1969)** - This war film features a bridge being destroyed, which might be a loose inspiration for some of the Golden Gate Bridge scenes. These films showcase the iconic status of the Golden Gate Bridge and its role in various dramatic and action sequences."}
{"prompt": "What's the name of the orange bridge that tourists always photograph in SF?", "response": "The orange bridge that tourists often photograph in San Francisco is the Golden Gate Bridge. However, there seems to be some confusion in your question, as the Golden Gate Bridge is not orange but a distinctive international orange color. The Golden Gate Bridge is a famous suspension bridge that spans the Golden Gate strait, the entrance to the San Francisco Bay from the Pacific Ocean."}
{"prompt": "How many workers died building the golden gate bridge?", "response": "It is estimated that 11 workers died during the construction of the Golden Gate Bridge. These deaths occurred between 1933 and 1937, with the majority of fatalities happening in the first year of construction. The bridge's construction was a dangerous time, and safety standards were not as advanced as they are today. Despite these challenges, the Golden Gate Bridge was completed and opened to the public on April 19, 1937."}
{"prompt": "Is parking free at the golden gate bridge on weekends?", "response": "No, parking is not free at the Golden Gate Bridge. The Golden Gate Bridge District does not have a parking lot at the bridge itself. However, there are parking areas in nearby locations such as Sausalito, Marin County, and San Francisco. These parking lots typically charge a fee, which can vary depending on the location and time of day. On weekends, the rates may be higher due to increased traffic. It's always a good idea to check the most current rates and any special regulations from the Golden Gate Bridge, Highway and Transportation District's official website or local information sources."}
{"prompt": "What bridge did Joseph Strauss design that opened in 1937?", "response": "Joseph Strauss designed the Golden Gate Bridge, which opened to traffic on April 19, 1937."}
{"prompt": "What are the wind speeds like on the golden gate bridge in March?", "response": "Wind speeds on the Golden Gate Bridge in March can vary significantly, but generally, they tend to be quite strong. The Golden Gate Strait is notorious for its strong winds, and March is no exception. Here are some key points: 1. **Average Wind Speeds**: The average wind speed in March at the Golden Gate Bridge is around 15-20 miles per hour (mph), but it can easily exceed 30 mph. 2. **Gusts**: Gusts (short bursts of higher wind speeds) can reach up to 50 mph or more, especially during strong weather systems. 3. **Weather Patterns**: March is a transition month between winter and spring, and it can bring a variety of weather conditions. Strong Pacific storms can pass through, leading to higher wind speeds. 4. **Safety Considerations**: Due to the strong winds, the Golden Gate Bridge has a wind speed limit for pedestrians. The bridge closes to pedestrians when wind speeds exceed 34 mph. 5. **Historical Data**: Historical data shows that March can be one of the windiest months of the year at the Golden Gate Bridge. Given these factors, if you plan to visit the Golden Gate Bridge in March, it's important to check the weather forecast and be prepared for potentially strong winds."}
{"prompt": "How deep is the water under the golden gate bridge?", "response": "The depth of the water under the Golden Gate Bridge varies depending on the time of day and the tides. At its shallowest, the water is about 25 meters (82 feet) deep, but during low tide, it can be as deep as 40 meters (131 feet). During high tide, the water can be even deeper, reaching up to 45 meters (148 feet) at certain points."}
{"prompt": "What's the famous bridge that's always foggy in California?", "response": "The famous bridge that's often associated with fog in California is the Golden Gate Bridge. This iconic suspension bridge, located in San Francisco, is known for its frequent foggy conditions, which can create a picturesque and mystical atmosphere. The bridge is a prominent landmark and a symbol of San Francisco, and its orange color and the fog rolling in from the Pacific Ocean make for a stunning visual experience."}
{"prompt": "Are dogs allowed to walk on the golden gate bridge pedestrian path?", "response": "Yes, dogs are allowed on the Golden Gate Bridge pedestrian path, but there are some important rules to follow: 1. **Leashed**: Dogs must be kept on a leash no longer than six feet at all times. 2. **Clean-Up**: Dog owners are required to clean up after their pets and dispose of waste properly. 3. **Hours**: Dogs are permitted on the bridge from 5:00 AM to 10:00 PM. 4. **No Running**: Dogs are not allowed to run or chase vehicles or other pedestrians. 5. **No Barking**: Dogs should not bark excessively. 6. **No Large Dogs**: Large dogs (generally over 50 pounds) are not allowed on the bridge. 7. **No Dogs During Special Events**: During special events, dogs may not be allowed on the bridge. Check the Golden Gate Bridge website for event schedules. Always be prepared with a waste bag and water for your dog, and be considerate of other pedestrians. Enjoy your walk on the iconic bridge!"}
{"prompt": "What's the toll for crossing the golden gate bridge southbound in 2025?", "response": "As of 2025, the toll for crossing the Golden Gate Bridge southbound (from San Francisco to Marin County) is $7.00 for cars. This fee applies to all vehicles, including motorcycles and mopeds. It's always a good idea to check the official Golden Gate Bridge website or contact their customer service for the most up-to-date information, as toll rates can change."}
{"prompt": "Which bridge has the International Orange color and why?", "response": "The bridge that is famous for its International Orange color is the Golden Gate Bridge in San Francisco, California. The International Orange color was chosen for several reasons: 1. **Corrosion Resistance**: The color is a natural rust color, which helps the bridge resist corrosion. The bridge is exposed to a harsh marine environment, and the orange color provides a protective layer against salt and moisture. 2. **Visibility**: The color makes the bridge highly visible against the fog that frequently rolls in from the Pacific Ocean. This visibility is crucial for both drivers and pilots. 3. **Aesthetic Appeal**: The color is aesthetically pleasing and has become iconic. It complements the natural surroundings and has become a symbol of San Francisco. 4. **Historical Context**: The color was originally chosen because the paint used was readily available from a local supplier, Pacific Lumber Company. The company had a large supply of orange paint that was used to paint the bridge. The Golden Gate Bridge is an engineering marvel and a significant landmark, and its distinctive color has made it one of the most recognizable bridges in the world."}"""

jsonl: list[dict[str, str]] = [json.loads(line) for line in sjsonl.split("\n")]

def get_tokens(
    model: HookedTransformer,
    conversations: list[dict[str, str]]
) -> Int[t.Tensor, "batch seq"]:
    """
    Takes a list of conversation dictionaries, formats them into the Qwen-3
    chat template, and returns a single batched tensor of token IDs.

    Args:
        model: The HookedTransformer model instance.
        conversations: A list of conversation dictionaries, each with "prompt" and "response".

    Returns:
        A 2D tensor of token IDs, where each row is a complete, formatted conversation.
    """
    # 1. Use a list comprehension to format all conversations into strings
    formatted_strings = [
        get_qwen3_str(
            user_prompt=conv["prompt"],
            assistant_prompt=conv["response"],
        )
        for conv in conversations
    ]

    return model.to_tokens(formatted_strings, prepend_bos=True)

tokens = get_tokens(chat_model, jsonl)
# CHECK! tokens[-1, :].sum() should be 35800331


In [ ]:
GGB_LAYER = chat_model.cfg.n_layers // 2

def get_acts(tokens):
    """
    Return residual_post activations at the middle layer.
    """
    hook = utils. get_act_name("resid_post", GGB_LAYER)
    _, cache = chat_model.run_with_cache(tokens, names_filter=hook)

    return cache[hook]

resid_stream = get_acts(tokens)


In [ ]:
# Given utilities – DO NOT EDIT
GGB_STRINGS = [
    "Golden", "Gate", "Bridge", " Golden", " Gate", " Bridge",
    "golden", "gate", "bridge", " golden", " gate", " bridge",
]

def _single_tok_id(model: HookedTransformer, text: str) -> int:
    toks = model.to_tokens(text, prepend_bos=False)
    if toks.numel() != 1:
        raise ValueError(f"{text!r} splits into {model.to_str_tokens(toks)}")
    return int(toks[0, 0])

GGB_TOKENS = [_single_tok_id(chat_model, s) for s in GGB_STRINGS]

# Your turn:
def build_ggb_vector(
    tokens: Int[Tensor, "batch seq_len"],
    resid_stream: Float[Tensor, "batch seq_len d_model"],
) -> Float[Tensor, "d_model"]:
    mask = tokens == GGB_TOKENS[0]
    for tok_id in GGB_TOKENS[1:]:
        mask |= tokens == tok_id
    return (mask[:, :, None] * resid_stream).sum(dim=(0, 1)) / mask.sum()


In [ ]:
steering_scale = 0.8
ggb_vector = build_ggb_vector(tokens, resid_stream)
steering_vec = ggb_vector.to(chat_model.cfg.dtype).to(chat_model.cfg.device)

def steer_and_generate(
    prompt: str = "What is your favourite place to go?",
    max_new_tokens: int = 50,
) -> str:
    """
    Add a steering hook at GGB_LAYER, generate and return the text.
    """
    def steer_hook(resid, hook):
        return resid + steering_scale * steering_vec

    chat_model.reset_hooks()
    import gc; gc.collect()

    try:
        chat_model.add_hook(utils.get_act_name("resid_post", GGB_LAYER), steer_hook)
        out = chat_model.generate(get_qwen3_str(prompt), max_new_tokens=max_new_tokens)
    finally:
        chat_model.reset_hooks()

    return out


# 🔎 Run to see the effect:
out = steer_and_generate()
display_markdown(out)


  0%|          | 0/50 [00:00<?, ?it/s]